# Hente hemmeligheter fra secret manager

Denne guiden er inspirert av https://docs.knada.io/prosessere-data/notebook_general/#credentials-og-hemmeligheter.

Første steg er å installere secret manager-pakken på din notebookserver. Dette steget trengs bare første gang.  
Åpne en terminal og kjør `pip install google-cloud-secret-manager`

Deretter må du oppdatere dine google credentials ved å kjøre `gcloud auth login --update-adc` og følge instruksjonen du får for å logge på din NAV-googlekonto.

Importer secret manager-pakken

In [ ]:
from google.cloud import secretmanager

Opprett en klient

In [ ]:
secrets = secretmanager.SecretManagerServiceClient()

Aksesser din hemmelighet - bytt ut variablene til dine egne

In [ ]:
project_name = "knada-gcp"
secret_name = "min-hemmelighet"
version = "latest"

resource_name = f"projects/{project_name}/secrets/{secret_name}/versions/{version}"
secret = secrets.access_secret_version(name=resource_name)

Les hemmelighetene dine inn i en dictionary

In [ ]:
secrets = dict([line.split(":") for line in secret.payload.data.decode('UTF-8').splitlines()])

# Koble til en postgres database med hemmelighetene dine

In [ ]:
import psycopg2

import pandas as pd

from sqlalchemy import create_engine

In [ ]:
method = "postgresql+psycopg2"
host = "10.53.15.223:5432"
db_name = "ufore-sommerstudenter"

alchemyEngine = create_engine(f'{method}://{secrets["UFORE_SOMMER_USER"]}:{secrets["UFORE_SOMMER_PASSWORD"]}@{host}/{db_name}')
dbConnection = alchemyEngine.connect()

In [ ]:
pd.read_sql('select * from ufore;', dbConnection)